In [45]:
import pandas as pd
from sklearn.cluster import KMeans
import folium
from folium.plugins import HeatMap
import matplotlib.pyplot as plt

In [46]:
# Base directory where your data is located
base_directory = '/Users/nt/Desktop/DS2500/'

# Define file paths
getting_files = [base_directory + f'{year}.csv' for year in range(2015, 2024)]

# Load data from 2015 to 2023
crime_data = {str(year): pd.read_csv(files) for year, files in zip(range(2015, 2024), getting_files)}

# Combines all data from 2015 to 2023
crime_data_combined = pd.concat([crime_data[str(year)] for year in range(2015, 2024)])
crime_data_combined = crime_data_combined.dropna(subset=['Location']) 

# Converting the location column strings to tuples
crime_data_combined['Location'] = crime_data_combined['Location'].apply(eval)

# Filtering the coordiantes to be in range for Boston
filtered_data = crime_data_combined[
    crime_data_combined['Location'].apply(lambda x: 42.22 <= x[0] <= 42.4 and -71.19 <= x[1] <= -71.0)]

/var/folders/25/k80xgfp945v9hr0_xm4s6h2c0000gn/T/ipykernel_34426/784972636.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_data = {str(year): pd.read_csv(files) for year, files in zip(range(2015, 2024), getting_files)}
/var/folders/25/k80xgfp945v9hr0_xm4s6h2c0000gn/T/ipykernel_34426/784972636.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_data = {str(year): pd.read_csv(files) for year, files in zip(range(2015, 2024), getting_files)}
/var/folders/25/k80xgfp945v9hr0_xm4s6h2c0000gn/T/ipykernel_34426/784972636.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_data = {str(year): pd.read_csv(files) for year, files in zip(range(2015, 2024), getting_files)}
/var/folders/25/k80xgfp945v9hr0_xm4s6h2c0000gn/T/ipykernel_34426/784972636.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option

In [47]:

# Train KMeans model
kmeans_model = KMeans(n_clusters=15, random_state=0, n_init="auto").fit(filtered_data['Location'].tolist())

# Predict clusters for the dataset
predicted_data = filtered_data.copy()
predicted_data['Cluster'] = kmeans_model.predict(filtered_data['Location'].tolist())

# Count occurrences of each predicted cluster
clusters = predicted_data['Cluster'].value_counts()

# Results in top 10 clusters with the highest count
top_10_clusters = clusters.nlargest(10)


In [48]:

# Finding the district and street names for the top clusters
top_10_streets = []
top_10_districts = []
for cluster in top_10_clusters.index:
    cluster_data = predicted_data[predicted_data['Cluster'] == cluster]
    street = cluster_data['STREET'].value_counts().idxmax()
    district = cluster_data['DISTRICT'].value_counts().idxmax()
    top_10_streets.append(street)
    top_10_districts.append(district)

# Printing the top 10 streets and its districts
print("Top 10 Predicted Dangerous Streets for 2024:")
for i, (street, district) in enumerate(zip(top_10_streets, top_10_districts), start=1):
    print(f"{i}. Street: {street}, District: {district}")
    

Top 10 Predicted Dangerous Streets for 2024:
1. Street: HARRISON AVE, District: B2
2. Street: GIBSON ST, District: C11
3. Street: WASHINGTON ST, District: A1
4. Street: BLUE HILL AVE, District: B2
5. Street: BLUE HILL AVE, District: B3
6. Street: WASHINGTON ST, District: E13
7. Street: BOYLSTON ST, District: D4
8. Street: NEW SUDBURY ST, District: A1
9. Street: WASHINGTON ST, District: D14
10. Street: COLUMBIA RD, District: C11


In [49]:
# Settting up a map of Boston
boston = folium.Map(location=[42.3601, -71.0589], zoom_start=12)

# Displays the top streets on to the map
for street, district in zip(top_10_streets, top_10_districts):
    street_data = predicted_data[predicted_data['STREET'] == street]
    location = street_data.iloc[0]['Location']
    
    # Information that will be shown in the info-sign
    icon_sign = f"Street: {street}<br>District: {district}"
    folium.Marker(location, popup=icon_sign, icon=folium.Icon(color='red', icon='info-sign')).add_to(boston)

# Displays Boston map
boston